### testing if the code is broken (it is)

In [1]:
import sys,os,os.path

sys.path.append("../../")   # cite IC from parent directory
                            # NOTE if you can't import IC stuff, its because of the
                            # above line
#sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['ICTDIR']='/home/e78368jw/Documents/NEXT_CODE/IC'

import matplotlib.pyplot as plt
import pandas as pd
import numpy  as np
import tables as tb
import IC.invisible_cities.io.dst_io                           as     dstio
import IC.invisible_cities.io.mcinfo_io as mcio
from    IC.invisible_cities.core.core_functions   import shift_to_bin_centers
import iminuit,probfit

import scipy.special as special
from scipy.stats import skewnorm
from scipy.optimize import curve_fit

from scipy.integrate import quad


#adding here ROI
emin = 1.55
emax = 1.65

In [2]:
# just functions 

def cut_effic(df1, df2, verbose = False):
    '''
    Prints efficiency of cuts for singular cut
    df1 -> cut df
    df2 -> initial df
    '''
    length_1 = df1['event'].nunique()
    length_2 = df2['event'].nunique()
    efficiency = ((length_1/length_2)*100)
    

    if (verbose == True):
        print("Efficiency: {:.2f} %".format(efficiency))
        print(("Events in reduced dataframe: {}\nEvents in initial dataframe: {}").format(len(df1), len(df2)))

    return efficiency




def plot_hist(df, column = 'energy', binning = 20, title = "Energy plot", output = True, fill = True, label = 'default', x_label = 'energy (MeV)', range = 0, data = False, save_title = 'False'):
    '''
    Print a histogram of energy from our dataframe,.
    '''
    # for simplicity/readability, scoop out the relevant columns from the pandas dataframe.
    energy_vals = df[column].to_numpy()

    if (range==0):
        range = (np.min(energy_vals), np.max(energy_vals))

    # control viewing of hist
    if (fill == True):
        cnts, edges, patches = plt.hist(energy_vals, bins = binning, label = label, range = range)
    else:
        cnts, edges, patches = plt.hist(energy_vals, bins = binning, label = label, histtype='step', linewidth = 2, range = range)
    plt.title(title)
    plt.ylabel("events")
    plt.xlabel(x_label)

    if (save_title == 'False'):
        print("")
    else:
        plt.savefig(save_title)
        

    if (output==True):
        plt.show()
    if (data==True):
        return (cnts, edges, patches)
    else:
        return




def fiducial_track_cut_2(df, lower_z = 20, upper_z = 1195, r_lim = 472, verbose = False):
    '''
    Produces fiducial track cuts while removing all events that have outer fiducial tracks
    '''
    # create lists of outer_fiduc entries
    z_df_low = df[(df['z_min'] <= lower_z)]
    z_df_up = df[(df['z_max'] >= upper_z)]
    r_df = df[(df['r_max'] >= r_lim)]

    # scrape the events
    low_list = (z_df_low['event'].to_numpy())
    up_list = (z_df_up['event'].to_numpy())
    r_list = (r_df['event'].to_numpy())

    # apply the filter to remove all events that fall in outer fiduc
    df1 = df[~df['event'].isin(low_list)]
    df2 = df1[~df1['event'].isin(up_list)]
    df3 = df2[~df2['event'].isin(r_list)]

    if (verbose == True):
        print("Cutting events around fiducial volume related to:\nZ range between {} and {}\nRadius range < {}".format(lower_z, upper_z, r_lim))


    return df3




def one_track_cuts(df, verbose = False):
    '''
    Remove events with more than one track
    THERE IS A COLUMN WITH THIS INFO IN IT, CALCULATING IT IS UNNECESSARY
    '''
    # 1-track event counter
    event_counts = df.groupby('event').size()
    #print(event_counts[:5]) # showing that you see how many 
                            #  trackIDs there are per event
    one_track = event_counts[event_counts == 1].index

    # filter dataframe
    one_track_events = df[df['event'].isin(one_track)]
    

    if (verbose == True):
        print("Removing events with more than one track.")
        print("Events with one track: {}".format(one_track))
        display(one_track_events.head())
    

    return one_track_events

def one_track_cuts_fast(df, verbose = False):
    '''
    Do it by just selection of events with 1 track
    '''

    one_track_df = df['numb_of_tracks']


def overlapping_cuts(df, verbose = False):
    '''
    Remove all events with energy overlap != 0
    '''

    ovlp_remove = df[df['ovlp_blob_energy']==0]

    if (verbose==True):
        print("Removing overlapping blobs...")

    return ovlp_remove




def energy_cuts(df, lower_e = 1.55, upper_e = 1.65, verbose = False):
    '''
    Apply cuts around the relevant energy
    '''
    filt_e_df = df[(df['energy'] >= lower_e) & (df['energy'] <= upper_e)]

    if (verbose == True):
        print("Cutting energy events around {} & {} keV".format(lower_e, upper_e))

    return filt_e_df




def len_events(df):
    '''
    Returns the number of unique events as len(df) doesn't work in this case
    '''
    length_1 = df['event'].nunique()
    return length_1


# useful to normalize histograms
def get_weights(data, norm):
    if norm:
        return np.repeat(1.0/len(data), len(data))
    else:
        return np.repeat(1.0, len(data))


def plot_volume_hists(df):

    plot_hist(df, column = 'x_min', output = False, label = 'x_min', fill = False, x_label = 'position (mm)')
    plot_hist(df, column = 'x_max', title = 'x plot for full volume', output = False, label = 'x_max', fill = False, x_label = 'position (mm)')
    plt.legend(loc='upper left')
    plt.show()

    plot_hist(df, column = 'y_min', output = False, label = 'y_min', fill = False, x_label = 'position (mm)')
    plot_hist(df, column = 'y_max', title = 'y plot for full volume', output = False, label = 'y_max', fill = False, x_label = 'position (mm)')
    plt.legend(loc='upper left')
    plt.show()

    plot_hist(df, column = 'z_min', output = False, label = 'z_min', fill = False, x_label = 'position (mm)')
    plot_hist(df, column = 'z_max', title = 'z plot for full volume', output = False, label = 'z_max', fill = False, x_label = 'position (mm)')
    plt.legend(loc='upper left')
    plt.show()

def read_MC_tracks(folder_path):
    '''
    Read in data from isaura events (MC and tracks)
    Checks the event mapping and adjusts if as expected
    '''

    file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # NOTE Break this section up, its annoying like this.
    dfs = []
    df_trs = []
    df_ems = []
    # create massive dataframe with all of them
    for file in file_names:
        file_path = folder_path + file
        df = dstio.load_dst(file_path, 'Tracking', 'Tracks')
        dfs.append(df)

        # include MC particles (boooo takes ages)

        # collecting the correct components of the file, not exactly sure how this works
        df_ps = pd.read_hdf(file_path, 'MC/particles')

        # collecting event map
        df_em = mcio.load_eventnumbermap(file_path).set_index('nexus_evt')
        df_trs.append(df_ps)
        df_ems.append(df_em)

    tracks = pd.concat(dfs, axis=0, ignore_index=True)

    particles = pd.concat(df_trs, ignore_index=True)
    eventmap = pd.concat([dt for dt in df_ems])
    # create particle list also

    # check that the event map is as expected and modify to match MC to true data
    eventmap_reset = eventmap.reset_index()
    if not (eventmap_reset['nexus_evt'] * 2 == eventmap_reset['evt_number']).all():
        print("Event mapping no longer accurate between nexus and isaura events.\nPlease look at the eventmap object again")
    else:
        particles['event_id'] = particles['event_id'] * 2 
    
    # return 
    return (tracks, particles)


def remove_low_E_events(df, energy_limit = 0.05):
    '''
    Remove low energy tracks, add their energy back to the first
    track and then update 'numb_of_tracks' to be up to date
    '''

    tracks_test = df.copy(deep=True)

    # take events with lower than 50 keV, 0.05 MeV
    condition = (tracks_test.energy < energy_limit)
    summed_df = tracks_test[condition].groupby('event')['energy'].sum().reset_index()

    # merge these as a new column
    merged_df = pd.merge(tracks_test, summed_df, on='event', suffixes=('', '_sum'))

    # add this summed energy to first column
    merged_df['energy'] = merged_df.apply(lambda row: (row['energy'] + row['energy_sum']) if row.name == merged_df[merged_df['event'] == row['event']].index[0] else row['energy'], axis=1)

    # drop energy sum column
    result_df = merged_df.drop('energy_sum', axis = 1)

    # then remove all tracks below the energy threshold
    condition_upper = (result_df.energy > energy_limit)
    remove_low_E = result_df[condition_upper]

    # count the number of events identified with unique event, and change numb_of_tracks to reflect this
    event_counts = remove_low_E['event'].value_counts(sort = False)

    # apply this to numb_of_tracks
    remove_low_E['numb_of_tracks'] = remove_low_E['event'].map(event_counts)

    return remove_low_E


def apply_all_cuts(tracks, verbose = False, low_e_cut_plot = 'False'):
    '''
    Apply all relevant cuts
    
    returns efficiency table and the fully cut data
    
    
    This function makes me physically ill,
    but was made in a rush.
    Please rework this, for the love of god
    '''

    # Efficiency calculation
    cut_names = []
    rel_cut_effics = []
    abs_cut_effics = []
    cut_events = []


    # no cuts
    cut_names.append("No cuts")
    rel_cut_effics.append(100)
    abs_cut_effics.append(100)
    # number of events
    cut_events.append(len_events(tracks))


    cut_names.append("Fiducial Cuts")

    # make fiducial cuts
    fiducial_rel = fiducial_track_cut_2(tracks, lower_z = 20, upper_z=1185, r_lim = 475, verbose = False)

    fiducial_abs = fiducial_track_cut_2(tracks, lower_z = 20, upper_z=1185, r_lim = 475, verbose = False)

    # make efficiency calculation
    if (verbose == True):
        print("Fiducial track cut")
        print("==================")
        print("Relative Cut efficiency:")
        ef = cut_effic(fiducial_rel, tracks, verbose = True)
    else:
        ef = cut_effic(fiducial_rel, tracks, verbose = False)
    rel_cut_effics.append(ef)
    cut_events.append(len_events(fiducial_rel))

    if (verbose == True):
        print('Absolute Cut efficiency:')
        ef = cut_effic(fiducial_abs, tracks, verbose = True)
    else:
        ef = cut_effic(fiducial_abs, tracks, verbose = False)
    abs_cut_effics.append(ef)


    low_e_cut_tracks_rel = remove_low_E_events(fiducial_rel)
    low_e_cut_tracks_abs = remove_low_E_events(tracks)


    # plot low e cut tracks if you want
    if (low_e_cut_plot == 'False'):
        print("")
    else:

        track_energy = low_e_cut_tracks_abs.energy
        track_no = low_e_cut_tracks_abs.numb_of_tracks
        # normalise
        weights = get_weights(track_energy, True)
        # clearing just in case
        plt.clf()
        # plot
        plt.hist2d(track_energy, track_no, weights = weights, bins=(100, tracks['numb_of_tracks'].max()), cmin=0.001)
        plt.title("Track energies: Low Pressure w/ low E cut: " + str(low_e_cut_plot))
        plt.xlabel('Energy (MeV)')
        plt.ylabel('Number of tracks')
        plt.xlim([0,2.0])
        plt.colorbar()

        plt.savefig("trk_E_vs_no_trk_low_E.png")


        if (verbose == True):
            plt.show()


    # low energy cut
    cut_names.append("low E cuts")

    # make efficiency calculation
    if (verbose == True):
        print("Low E cut")
        print("==================")
        print("Relative Cut efficiency:")
        ef = cut_effic(low_e_cut_tracks_rel, fiducial_rel, verbose = True)
    else:
        ef = cut_effic(low_e_cut_tracks_rel, fiducial_rel)

    # add efficiency calculation
    rel_cut_effics.append(ef)
    cut_events.append(len_events(low_e_cut_tracks_rel))

    if (verbose == True):
        print("==================")
        print("Absolute Cut efficiency:")
        ef = cut_effic(low_e_cut_tracks_abs, tracks)
    else:
        ef = cut_effic(low_e_cut_tracks_abs, tracks)

    abs_cut_effics.append(ef)

    # relative single track
    cut_names.append("One track cut")
    one_track_rel = one_track_cuts(low_e_cut_tracks_rel, verbose = False)

    # events are relative, as absolute efficiency lets you figure out events from the beginning# absolute
    one_track_abs = one_track_cuts(tracks)

    ## sanity check here
    #print(len_events(one_track_rel), len_events(one_track_abs), len_events(tracks))

    # relative
    if (verbose == True):
        print("Single track cut")
        print("================")
        print("Relative Cut efficiency:")
        ef = cut_effic(one_track_rel, low_e_cut_tracks_rel, verbose = True)
    else:
        ef = cut_effic(one_track_rel, low_e_cut_tracks_rel, verbose = False)
    rel_cut_effics.append(ef)
    cut_events.append(len_events(one_track_rel))

    # absolute
    if (verbose == True):
        print("Absolute Cut efficiency:")
        ef = cut_effic(one_track_abs, tracks, verbose = True)
    else:
        ef = cut_effic(one_track_abs, tracks, verbose = False)
    abs_cut_effics.append(ef)


    # apply cuts
    ovlp_rel = overlapping_cuts(one_track_rel)
    ovlp_abs = overlapping_cuts(tracks)

    cut_names.append("Blob overlap cuts")

    
    # relative
    if (verbose == True):
        print("Blob overlap cut")
        print("================")
        print("Relative Cut efficiency:")
        ef = cut_effic(ovlp_rel, one_track_rel, verbose = True)
    else:
        ef = cut_effic(ovlp_rel, one_track_rel, verbose = False)
    rel_cut_effics.append(ef)
    cut_events.append(len_events(ovlp_rel))


    # absolute
    if (verbose == True):
        print("Absolute Cut efficiency:")
        ef = cut_effic(ovlp_abs, tracks, verbose = True)
    else:    
        ef = cut_effic(ovlp_abs, tracks, verbose = False)
    abs_cut_effics.append(ef)


    # energy cuts
    ecut_rel = energy_cuts(ovlp_rel)
    ecut_abs = energy_cuts(tracks)

    cut_names.append("Energy cuts")

    # relative
    if (verbose == True):
        print("Energy cut")
        print("================")
        print("Relative Cut efficiency:")
        ef = cut_effic(ecut_rel, ovlp_rel, verbose = True)
    else:
        ef = cut_effic(ecut_rel, ovlp_rel, verbose = False)
    rel_cut_effics.append(ef)
    cut_events.append(len_events(ecut_rel))


    # absolute
    if (verbose == True):    
        print("Absolute Cut efficiency:")
        ef = cut_effic(ecut_abs, tracks, verbose = True)
    else:
        ef = cut_effic(ecut_abs, tracks, verbose = False)
    abs_cut_effics.append(ef)


    efficiencies = pd.DataFrame({'Cut': cut_names,
                             'Relative Efficiency': rel_cut_effics,
                             'Relative Events': cut_events,
                             'Single Cut Efficiency': abs_cut_effics
                             })

    if (verbose == True):
        display(efficiencies)
        print("Single Cut Efficiency: each cut on the original unmodified data set\nRelative Efficiency: each cut efficiency wrt the previous cut")


    # return it
    return (ecut_rel, efficiencies)


In [3]:
# list of folder paths for the different voxelisations
# NOTE If you want to change this, should do it here


folder_path = []
# port 1a, all voxelisations
# folder_path.append(['../LPR_rerun_voxel/isaura_15x15x15mm/PORT_1a/prod/isaura/','../LPR_rerun_voxel/isaura_10x10x10mm/PORT_1a/prod/isaura/', '../LPR_rerun_voxel/isaura_10x10x5mm/PORT_1a/prod/isaura/'])
# # port 1b, all voxelisations
# folder_path.append(['../LPR_rerun_voxel/isaura_15x15x15mm/PORT_1b/prod/isaura/','../LPR_rerun_voxel/isaura_10x10x10mm/PORT_1b/prod/isaura/', '../LPR_rerun_voxel/isaura_10x10x5mm/PORT_1b/prod/isaura/'])
# # port 2a, all voxelisations
# folder_path.append(['../LPR_rerun_voxel/isaura_15x15x15mm/PORT_2a/prod/isaura/','../LPR_rerun_voxel/isaura_10x10x10mm/PORT_2a/prod/isaura/', '../LPR_rerun_voxel/isaura_10x10x5mm/PORT_2a/prod/isaura/'])
# # port 2b, all voxelisations
# folder_path.append(['../LPR_rerun_voxel/isaura_15x15x15mm/PORT_2b/prod/isaura/','../LPR_rerun_voxel/isaura_10x10x10mm/PORT_2b/prod/isaura/', '../LPR_rerun_voxel/isaura_10x10x5mm/PORT_2b/prod/isaura/'])

folder_path.append(['../LPR_rerun_voxel/isaura_10x10x10mm/PORT_1a/prod/isaura/', '../LPR_rerun_voxel/isaura_10x10x10mm/PORT_1b/prod/isaura/', '../LPR_rerun_voxel/isaura_10x10x10mm/PORT_2a/prod/isaura/', '../LPR_rerun_voxel/isaura_10x10x10mm/PORT_2b/prod/isaura/'])


# flatten
folder_path = [x for xs in folder_path for x in xs]


folder_title = [ '10x10x10mm_Port_1a',   
                 '10x10x10mm_Port_1b',
                 '10x10x10mm_Port_2a', 
                 '10x10x10mm_Port_2b']



In [4]:
# read in some data
tracks, particles = read_MC_tracks(folder_path[0])
tracks_test = tracks.copy(deep=True)

In [5]:
# break down exactly whats going on in the function for low cuts here
display(tracks)
display(tracks.head(10))
print(tracks['event'].nunique())

,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
0,5080000,0,1.051154,121.454061,104,11063,2,368.675,160.675,334.439758,...,358.948175,381.376657,256.794897,408.658585,0.483891,0.015104,0.000000,9.857143,9.928571,9.619794
1,5080000,1,0.013352,9.928571,2,71,2,388.675,270.675,440.460083,...,442.236380,390.287922,283.647294,446.131930,0.013352,0.013352,0.013352,9.857143,9.928571,9.619794
2,5080002,0,1.768218,136.325163,128,11033,5,-98.325,361.675,587.473511,...,613.110772,-6.622819,382.940215,589.840154,0.162713,0.073897,0.000000,9.750000,9.571429,9.902647
3,5080002,1,0.309468,112.683992,31,1829,5,52.675,348.675,498.257599,...,502.310940,54.698632,349.802676,526.248103,0.162199,0.021946,0.000000,9.750000,9.571429,9.902647
4,5080002,2,0.019542,13.772242,2,4,5,18.675,351.675,555.981140,...,563.299622,27.644104,359.876147,555.981140,0.019542,0.019542,0.019542,9.750000,9.571429,9.902647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258315,4060708,1,0.121964,14.142136,3,1244,3,184.675,346.675,305.074768,...,309.084361,199.819036,352.583988,308.110573,0.121964,0.121964,0.121964,10.000000,10.000000,9.859631
258316,4060708,2,0.033485,10.000000,2,144,3,153.675,366.675,386.938629,...,389.045441,162.594082,366.980910,390.887885,0.033485,0.033472,0.033472,10.000000,10.000000,9.859631
258317,4060710,0,0.572125,44.565517,36,7243,1,315.675,294.675,522.397217,...,532.700813,331.038243,299.916212,527.865748,0.279909,0.256440,0.002211,9.833333,9.428571,8.912888
258318,4060712,0,0.888838,109.999957,76,6895,2,-6.325,415.675,335.498718,...,352.800524,25.058429,476.143700,418.546023,0.688630,0.026967,0.000000,9.500000,8.875000,9.673330


,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
0,5080000,0,1.051154,121.454061,104,11063,2,368.675,160.675,334.439758,...,358.948175,381.376657,256.794897,408.658585,0.483891,0.015104,0.000000,9.857143,9.928571,9.619794
1,5080000,1,0.013352,9.928571,2,71,2,388.675,270.675,440.460083,...,442.236380,390.287922,283.647294,446.131930,0.013352,0.013352,0.013352,9.857143,9.928571,9.619794
2,5080002,0,1.768218,136.325163,128,11033,5,-98.325,361.675,587.473511,...,613.110772,-6.622819,382.940215,589.840154,0.162713,0.073897,0.000000,9.750000,9.571429,9.902647
3,5080002,1,0.309468,112.683992,31,1829,5,52.675,348.675,498.257599,...,502.310940,54.698632,349.802676,526.248103,0.162199,0.021946,0.000000,9.750000,9.571429,9.902647
4,5080002,2,0.019542,13.772242,2,4,5,18.675,351.675,555.981140,...,563.299622,27.644104,359.876147,555.981140,0.019542,0.019542,0.019542,9.750000,9.571429,9.902647
5,5080002,3,0.008122,0.000000,1,37,5,28.675,410.675,618.963989,...,620.147517,34.523942,415.043112,620.147517,0.008122,0.008122,0.008122,9.750000,9.571429,9.902647
6,5080002,4,0.000267,0.000000,1,4,5,163.675,337.675,502.410492,...,502.410492,164.174985,338.673183,502.410492,0.000267,0.000267,0.000267,9.750000,9.571429,9.902647
7,5080004,0,0.475175,33.603286,13,4183,4,12.675,-315.325,147.604263,...,151.930309,19.101046,-292.754173,153.639171,0.371749,0.309839,0.206413,10.000000,9.222222,7.255455
8,5080004,1,0.000826,0.000000,1,28,4,36.675,-259.325,154.847458,...,154.847458,38.559927,-256.434284,154.847458,0.000826,0.000826,0.000826,10.000000,9.222222,7.255455
9,5080004,2,0.000476,0.000000,1,21,4,22.675,-336.325,159.047852,...,159.047852,24.360200,-334.110790,159.047852,0.000476,0.000476,0.000476,10.000000,9.222222,7.255455


100009


In [8]:
# order rows by event number
test3 = tracks_test.sort_values(by='event', ascending=True).reset_index(drop=True)
display(test3)

,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
0,0,0,0.447855,42.439827,26,5417,2,354.675,-204.325,777.548096,...,786.653140,399.235626,-187.432439,783.612354,0.308510,0.087125,0.000000,9.705882,9.850000,9.125586
1,0,1,0.100168,13.828473,3,1009,2,247.675,-43.325,859.438843,...,865.805965,261.956932,-38.412266,862.666282,0.100168,0.100168,0.100168,9.705882,9.850000,9.125586
2,2,0,2.202431,500.014435,246,13569,3,124.675,-91.325,505.622162,...,566.336864,291.883283,168.340629,526.918150,0.907629,0.050585,0.000000,9.750000,9.794118,9.550124
3,2,1,0.153003,9.794118,2,1108,3,122.675,-121.325,402.714600,...,407.511482,126.735585,-108.348544,406.722532,0.153003,0.153003,0.153003,9.750000,9.794118,9.550124
4,2,2,0.000621,9.750000,2,50,3,128.675,-151.325,407.901794,...,407.901794,133.172353,-148.284251,407.901794,0.000621,0.000621,0.000621,9.750000,9.794118,9.550124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258315,5980684,2,0.182700,52.501054,23,1230,4,255.675,-107.325,711.367737,...,732.613773,262.059954,-82.691211,717.654256,0.101596,0.061803,0.000000,9.500000,9.777778,9.732688
258316,5980684,3,0.006471,0.000000,1,16,4,303.675,-87.325,758.615417,...,758.615417,304.435741,-85.291500,758.615417,0.006471,0.006471,0.006471,9.500000,9.777778,9.732688
258317,5980688,0,0.393076,26.695935,10,3755,3,110.675,253.675,475.163910,...,484.789052,132.683837,259.825503,479.657114,0.389572,0.378861,0.375490,9.757576,10.000000,9.575893
258318,5980688,1,0.081007,13.971767,4,914,3,403.675,233.675,491.896210,...,498.013538,415.611357,246.468925,497.533560,0.081007,0.081007,0.081007,9.757576,10.000000,9.575893


In [9]:
print(tracks.groupby('event').size())
display(tracks[tracks['event'] == 2])

event
0          2
2          3
4          7
6          2
8          2
          ..
5980678    2
5980680    3
5980682    3
5980684    4
5980688    3
Length: 100009, dtype: int64


,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
219199,2,0,2.202431,500.014435,246,13569,3,124.675,-91.325,505.622162,...,566.336864,291.883283,168.340629,526.918150,0.907629,0.050585,0.000000,9.75,9.794118,9.550124
219200,2,1,0.153003,9.794118,2,1108,3,122.675,-121.325,402.714600,...,407.511482,126.735585,-108.348544,406.722532,0.153003,0.153003,0.153003,9.75,9.794118,9.550124
219201,2,2,0.000621,9.750000,2,50,3,128.675,-151.325,407.901794,...,407.901794,133.172353,-148.284251,407.901794,0.000621,0.000621,0.000621,9.75,9.794118,9.550124


In [10]:
tracks_test = tracks.copy(deep=True)

condition = (tracks_test.energy < 0.05)
summed_df = tracks_test[condition].groupby('event')['energy'].sum().reset_index()

In [17]:
display(summed_df)

,event,energy
0,2,0.000621
1,4,0.022436
2,8,0.001634
3,12,0.004021
4,14,0.001492
...,...,...
54860,5980678,0.000264
54861,5980680,0.007079
54862,5980682,0.029189
54863,5980684,0.006471


In [11]:
# merge these as a new column
merged_df = pd.merge(tracks_test, summed_df, on='event', suffixes=('', '_sum'), how = 'left').fillna(0)

In [16]:
display(tracks.head(10))
display(merged_df.head(10))

,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
0,5080000,0,1.051154,121.454061,104,11063,2,368.675,160.675,334.439758,...,358.948175,381.376657,256.794897,408.658585,0.483891,0.015104,0.000000,9.857143,9.928571,9.619794
1,5080000,1,0.013352,9.928571,2,71,2,388.675,270.675,440.460083,...,442.236380,390.287922,283.647294,446.131930,0.013352,0.013352,0.013352,9.857143,9.928571,9.619794
2,5080002,0,1.768218,136.325163,128,11033,5,-98.325,361.675,587.473511,...,613.110772,-6.622819,382.940215,589.840154,0.162713,0.073897,0.000000,9.750000,9.571429,9.902647
3,5080002,1,0.309468,112.683992,31,1829,5,52.675,348.675,498.257599,...,502.310940,54.698632,349.802676,526.248103,0.162199,0.021946,0.000000,9.750000,9.571429,9.902647
4,5080002,2,0.019542,13.772242,2,4,5,18.675,351.675,555.981140,...,563.299622,27.644104,359.876147,555.981140,0.019542,0.019542,0.019542,9.750000,9.571429,9.902647
5,5080002,3,0.008122,0.000000,1,37,5,28.675,410.675,618.963989,...,620.147517,34.523942,415.043112,620.147517,0.008122,0.008122,0.008122,9.750000,9.571429,9.902647
6,5080002,4,0.000267,0.000000,1,4,5,163.675,337.675,502.410492,...,502.410492,164.174985,338.673183,502.410492,0.000267,0.000267,0.000267,9.750000,9.571429,9.902647
7,5080004,0,0.475175,33.603286,13,4183,4,12.675,-315.325,147.604263,...,151.930309,19.101046,-292.754173,153.639171,0.371749,0.309839,0.206413,10.000000,9.222222,7.255455
8,5080004,1,0.000826,0.000000,1,28,4,36.675,-259.325,154.847458,...,154.847458,38.559927,-256.434284,154.847458,0.000826,0.000826,0.000826,10.000000,9.222222,7.255455
9,5080004,2,0.000476,0.000000,1,21,4,22.675,-336.325,159.047852,...,159.047852,24.360200,-334.110790,159.047852,0.000476,0.000476,0.000476,10.000000,9.222222,7.255455


,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z,energy_sum
0,5080000,0,1.051154,121.454061,104,11063,2,368.675,160.675,334.439758,...,381.376657,256.794897,408.658585,0.483891,0.015104,0.000000,9.857143,9.928571,9.619794,0.013352
1,5080000,1,0.013352,9.928571,2,71,2,388.675,270.675,440.460083,...,390.287922,283.647294,446.131930,0.013352,0.013352,0.013352,9.857143,9.928571,9.619794,0.013352
2,5080002,0,1.768218,136.325163,128,11033,5,-98.325,361.675,587.473511,...,-6.622819,382.940215,589.840154,0.162713,0.073897,0.000000,9.750000,9.571429,9.902647,0.027931
3,5080002,1,0.309468,112.683992,31,1829,5,52.675,348.675,498.257599,...,54.698632,349.802676,526.248103,0.162199,0.021946,0.000000,9.750000,9.571429,9.902647,0.027931
4,5080002,2,0.019542,13.772242,2,4,5,18.675,351.675,555.981140,...,27.644104,359.876147,555.981140,0.019542,0.019542,0.019542,9.750000,9.571429,9.902647,0.027931
5,5080002,3,0.008122,0.000000,1,37,5,28.675,410.675,618.963989,...,34.523942,415.043112,620.147517,0.008122,0.008122,0.008122,9.750000,9.571429,9.902647,0.027931
6,5080002,4,0.000267,0.000000,1,4,5,163.675,337.675,502.410492,...,164.174985,338.673183,502.410492,0.000267,0.000267,0.000267,9.750000,9.571429,9.902647,0.027931
7,5080004,0,0.475175,33.603286,13,4183,4,12.675,-315.325,147.604263,...,19.101046,-292.754173,153.639171,0.371749,0.309839,0.206413,10.000000,9.222222,7.255455,0.001505
8,5080004,1,0.000826,0.000000,1,28,4,36.675,-259.325,154.847458,...,38.559927,-256.434284,154.847458,0.000826,0.000826,0.000826,10.000000,9.222222,7.255455,0.001505
9,5080004,2,0.000476,0.000000,1,21,4,22.675,-336.325,159.047852,...,24.360200,-334.110790,159.047852,0.000476,0.000476,0.000476,10.000000,9.222222,7.255455,0.001505


In [14]:
print(merged_df['event'].nunique())

100009


In [17]:
# so we know we have all the events now, merge as expected

merged_df['energy'] = merged_df.apply(lambda row: (row['energy'] + row['energy_sum']) if row.name == merged_df[merged_df['event'] == row['event']].index[0] else row['energy'], axis=1)

In [18]:
display(merged_df)

,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z,energy_sum
0,5080000,0,1.064505,121.454061,104,11063,2,368.675,160.675,334.439758,...,381.376657,256.794897,408.658585,0.483891,0.015104,0.000000,9.857143,9.928571,9.619794,0.013352
1,5080000,1,0.013352,9.928571,2,71,2,388.675,270.675,440.460083,...,390.287922,283.647294,446.131930,0.013352,0.013352,0.013352,9.857143,9.928571,9.619794,0.013352
2,5080002,0,1.796149,136.325163,128,11033,5,-98.325,361.675,587.473511,...,-6.622819,382.940215,589.840154,0.162713,0.073897,0.000000,9.750000,9.571429,9.902647,0.027931
3,5080002,1,0.309468,112.683992,31,1829,5,52.675,348.675,498.257599,...,54.698632,349.802676,526.248103,0.162199,0.021946,0.000000,9.750000,9.571429,9.902647,0.027931
4,5080002,2,0.019542,13.772242,2,4,5,18.675,351.675,555.981140,...,27.644104,359.876147,555.981140,0.019542,0.019542,0.019542,9.750000,9.571429,9.902647,0.027931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258315,4060708,1,0.121964,14.142136,3,1244,3,184.675,346.675,305.074768,...,199.819036,352.583988,308.110573,0.121964,0.121964,0.121964,10.000000,10.000000,9.859631,0.033485
258316,4060708,2,0.033485,10.000000,2,144,3,153.675,366.675,386.938629,...,162.594082,366.980910,390.887885,0.033485,0.033472,0.033472,10.000000,10.000000,9.859631,0.033485
258317,4060710,0,0.572125,44.565517,36,7243,1,315.675,294.675,522.397217,...,331.038243,299.916212,527.865748,0.279909,0.256440,0.002211,9.833333,9.428571,8.912888,0.000000
258318,4060712,0,0.889423,109.999957,76,6895,2,-6.325,415.675,335.498718,...,25.058429,476.143700,418.546023,0.688630,0.026967,0.000000,9.500000,8.875000,9.673330,0.000585


In [19]:
# drop energy sum column
result_df = merged_df.drop('energy_sum', axis = 1)

In [20]:
display(result_df)

,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
0,5080000,0,1.064505,121.454061,104,11063,2,368.675,160.675,334.439758,...,358.948175,381.376657,256.794897,408.658585,0.483891,0.015104,0.000000,9.857143,9.928571,9.619794
1,5080000,1,0.013352,9.928571,2,71,2,388.675,270.675,440.460083,...,442.236380,390.287922,283.647294,446.131930,0.013352,0.013352,0.013352,9.857143,9.928571,9.619794
2,5080002,0,1.796149,136.325163,128,11033,5,-98.325,361.675,587.473511,...,613.110772,-6.622819,382.940215,589.840154,0.162713,0.073897,0.000000,9.750000,9.571429,9.902647
3,5080002,1,0.309468,112.683992,31,1829,5,52.675,348.675,498.257599,...,502.310940,54.698632,349.802676,526.248103,0.162199,0.021946,0.000000,9.750000,9.571429,9.902647
4,5080002,2,0.019542,13.772242,2,4,5,18.675,351.675,555.981140,...,563.299622,27.644104,359.876147,555.981140,0.019542,0.019542,0.019542,9.750000,9.571429,9.902647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258315,4060708,1,0.121964,14.142136,3,1244,3,184.675,346.675,305.074768,...,309.084361,199.819036,352.583988,308.110573,0.121964,0.121964,0.121964,10.000000,10.000000,9.859631
258316,4060708,2,0.033485,10.000000,2,144,3,153.675,366.675,386.938629,...,389.045441,162.594082,366.980910,390.887885,0.033485,0.033472,0.033472,10.000000,10.000000,9.859631
258317,4060710,0,0.572125,44.565517,36,7243,1,315.675,294.675,522.397217,...,532.700813,331.038243,299.916212,527.865748,0.279909,0.256440,0.002211,9.833333,9.428571,8.912888
258318,4060712,0,0.889423,109.999957,76,6895,2,-6.325,415.675,335.498718,...,352.800524,25.058429,476.143700,418.546023,0.688630,0.026967,0.000000,9.500000,8.875000,9.673330


In [22]:
# then remove all tracks below the energy threshold
condition_upper = (result_df.energy > 0.05)
remove_low_E = result_df[condition_upper]

In [24]:
display(remove_low_E)
print(remove_low_E['event'].nunique())

,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
0,5080000,0,1.064505,121.454061,104,11063,2,368.675,160.675,334.439758,...,358.948175,381.376657,256.794897,408.658585,0.483891,0.015104,0.000000,9.857143,9.928571,9.619794
2,5080002,0,1.796149,136.325163,128,11033,5,-98.325,361.675,587.473511,...,613.110772,-6.622819,382.940215,589.840154,0.162713,0.073897,0.000000,9.750000,9.571429,9.902647
3,5080002,1,0.309468,112.683992,31,1829,5,52.675,348.675,498.257599,...,502.310940,54.698632,349.802676,526.248103,0.162199,0.021946,0.000000,9.750000,9.571429,9.902647
7,5080004,0,0.476680,33.603286,13,4183,4,12.675,-315.325,147.604263,...,151.930309,19.101046,-292.754173,153.639171,0.371749,0.309839,0.206413,10.000000,9.222222,7.255455
11,5080006,0,0.274088,26.444026,12,2404,2,94.675,449.675,725.018860,...,746.552973,106.717646,465.004488,726.796522,0.240083,0.170434,0.137690,9.250000,9.842105,9.772661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258312,4060706,1,0.067269,14.044372,3,834,3,-186.325,135.675,490.888611,...,493.630449,-173.840540,142.455140,492.813130,0.067269,0.067269,0.067269,9.981481,9.880000,9.442437
258314,4060708,0,0.317252,20.000000,6,2402,3,143.675,270.675,790.148376,...,800.718354,157.628521,287.500950,802.307167,0.283356,0.282692,0.282280,10.000000,10.000000,9.859631
258315,4060708,1,0.121964,14.142136,3,1244,3,184.675,346.675,305.074768,...,309.084361,199.819036,352.583988,308.110573,0.121964,0.121964,0.121964,10.000000,10.000000,9.859631
258317,4060710,0,0.572125,44.565517,36,7243,1,315.675,294.675,522.397217,...,532.700813,331.038243,299.916212,527.865748,0.279909,0.256440,0.002211,9.833333,9.428571,8.912888


100008


In [19]:
#merged_df = pd.merge(tracks_test, summed_df, on='event', suffixes=('', '_sum'))

In [23]:
display(merged_df)
print(merged_df['event'].nunique())

,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z,energy_sum
0,5080000,0,1.051154,121.454061,104,11063,2,368.675,160.675,334.439758,...,381.376657,256.794897,408.658585,0.483891,0.015104,0.000000,9.857143,9.928571,9.619794,0.013352
1,5080000,1,0.013352,9.928571,2,71,2,388.675,270.675,440.460083,...,390.287922,283.647294,446.131930,0.013352,0.013352,0.013352,9.857143,9.928571,9.619794,0.013352
2,5080002,0,1.768218,136.325163,128,11033,5,-98.325,361.675,587.473511,...,-6.622819,382.940215,589.840154,0.162713,0.073897,0.000000,9.750000,9.571429,9.902647,0.027931
3,5080002,1,0.309468,112.683992,31,1829,5,52.675,348.675,498.257599,...,54.698632,349.802676,526.248103,0.162199,0.021946,0.000000,9.750000,9.571429,9.902647,0.027931
4,5080002,2,0.019542,13.772242,2,4,5,18.675,351.675,555.981140,...,27.644104,359.876147,555.981140,0.019542,0.019542,0.019542,9.750000,9.571429,9.902647,0.027931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188196,4060708,0,0.283768,20.000000,6,2402,3,143.675,270.675,790.148376,...,157.628521,287.500950,802.307167,0.283356,0.282692,0.282280,10.000000,10.000000,9.859631,0.033485
188197,4060708,1,0.121964,14.142136,3,1244,3,184.675,346.675,305.074768,...,199.819036,352.583988,308.110573,0.121964,0.121964,0.121964,10.000000,10.000000,9.859631,0.033485
188198,4060708,2,0.033485,10.000000,2,144,3,153.675,366.675,386.938629,...,162.594082,366.980910,390.887885,0.033485,0.033472,0.033472,10.000000,10.000000,9.859631,0.033485
188199,4060712,0,0.888838,109.999957,76,6895,2,-6.325,415.675,335.498718,...,25.058429,476.143700,418.546023,0.688630,0.026967,0.000000,9.500000,8.875000,9.673330,0.000585


54865
